In [2]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import random
import json
import pickle
import numpy as np

import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', "!", ".", ","]

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(120, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Use the legacy optimizer
sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.model',hist)
print("done")



<ipython-input-3-11c3b16c1c3e>:54: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
7/7 [==============================] - 1s 4ms/step - loss: 1.9118 - accuracy: 0.1765
Epoch 2/200
7/7 [==============================] - 0s 3ms/step - loss: 1.9317 - accuracy: 0.2941
Epoch 3/200
7/7 [==============================] - 0s 3ms/step - loss: 1.8652 - accuracy: 0.2941
Epoch 4/200
7/7 [==============================] - 0s 3ms/step - loss: 1.8267 - accuracy: 0.2647
Epoch 5/200
7/7 [==============================] - 0s 3ms/step - loss: 1.7890 - accuracy: 0.3824
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 1.7908 - accuracy: 0.3235
Epoch 7/200
7/7 [==============================] - 0s 2ms/step - loss: 1.7010 - accuracy: 0.4412
Epoch 8/200
7/7 [==============================] - 0s 3ms/step - loss: 1.6672 - accuracy: 0.4118
Epoch 9/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5295 - accuracy: 0.4412
Epoch 10/200
7/7 [==============================] - 0s 2ms/step - loss: 1.4988 - accuracy: 0.4412
Epoch 11/200
7/7 [===========

In [4]:
from scipy.sparse.linalg._isolve.lsmr import result_type
from tensorflow.python.platform.tf_logging import ERROR
from nltk.translate.bleu_score import sentence_bleu
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from keras.models import load_model

lemmatizer=WordNetLemmatizer()
intents=json.loads(open('/content/intents.json').read())

words=pickle.load(open('/content/words.pkl','rb'))
classes=pickle.load(open('/content/classes.pkl','rb'))
model=load_model('chatbot_model.model')

def clean_up_sentense(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentense(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word==w:
        bag[i]=1
  return np.array(bag)

def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25
  results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

def get_response(intents_list,intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result
print("GO! bot is Running!")

while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)

GO! bot is Running!
hello
1/1 [==============================] - 0s 91ms/step
Hello!


KeyboardInterrupt: ignored